In [2]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

In [3]:
#loading the original dataset
data_path = "datasets/points_with_jobs_tele_ult_original.csv"
df = pd.read_csv(data_path, low_memory=False)

print("Rows:", df.shape[0])
print("Columns:", df.shape[1])
df.head()

Rows: 50259
Columns: 150


,timestamp,hostname,GPU,serial,is_failure,prev_xid48,next_xid48,hours_in_job,allocation_id,user_name,account,gpu_energy,gpu_usage,flag_gpumps,flag_gpudefault,flag_smt1,flag_smt2,flag_smt4,flag_nvme,flag_cpublink,flag_disableautonuma,flag_isolategpfs,flag_maximizegpfs,n_jobs_1h,jobhours_1h,n_jobs_6h,jobhours_6h,n_jobs_24h,jobhours_24h,n_jobs_120h,jobhours_120h,job_max_power,job_range_power,job_min_power,n_no_tele_6h,n_no_tele_1h,n_no_tele_15min,n_no_tele_5min,n_no_tele_1min,n_NA_power_6h,n_NA_core_temp_6h,n_NA_mem_temp_6h,n_NA_power_1h,n_NA_core_temp_1h,n_NA_mem_temp_1h,n_NA_power_15min,n_NA_core_temp_15min,n_NA_mem_temp_15min,n_NA_power_5min,n_NA_core_temp_5min,n_NA_mem_temp_5min,n_NA_power_1min,n_NA_core_temp_1min,n_NA_mem_temp_1min,power_fluct_6h,core_temp_fluct_6h,mem_temp_fluct_6h,power_fluct_1h,core_temp_fluct_1h,mem_temp_fluct_1h,power_fluct_15min,core_temp_fluct_15min,mem_temp_fluct_15min,power_fluct_5min,core_temp_fluct_5min,mem_temp_fluct_5min,power_fluct_1min,core_temp_fluct_1min,mem_temp_fluct_1min,power_fluct_lifetime,core_temp_fluct_lifetime,mem_temp_fluct_lifetime,power_avg_lifetime,core_temp_avg_lifetime,mem_temp_avg_lifetime,power_min_lifetime,power_max_lifetime,power_range_lifetime,core_temp_min_lifetime,core_temp_max_lifetime,core_temp_range_lifetime,mem_temp_min_lifetime,mem_temp_max_lifetime,mem_temp_range_lifetime,n_zero_power,last_zero_power,n_zero_core_temp,last_zero_core_temp,n_zero_mem_temp,last_zero_mem_temp,power_mean_6h,power_min_6h,power_max_6h,power_range_6h,core_temp_mean_6h,core_temp_min_6h,core_temp_max_6h,core_temp_range_6h,mem_temp_mean_6h,mem_temp_min_6h,mem_temp_max_6h,mem_temp_range_6h,power_mean_1h,power_min_1h,power_max_1h,power_range_1h,core_temp_mean_1h,core_temp_min_1h,core_temp_max_1h,core_temp_range_1h,mem_temp_mean_1h,mem_temp_min_1h,mem_temp_max_1h,mem_temp_range_1h,power_mean_15min,power_min_15min,power_max_15min,power_range_15min,core_temp_mean_15min,core_temp_min_15min,core_temp_max_15min,core_temp_range_15min,mem_temp_mean_15min,mem_temp_min_15min,mem_temp_max_15min,mem_temp_range_15min,power_mean_5min,power_min_5min,power_max_5min,power_range_5min,core_temp_mean_5min,core_temp_min_5min,core_temp_max_5min,core_temp_range_5min,mem_temp_mean_5min,mem_temp_min_5min,mem_temp_max_5min,mem_temp_range_5min,power_mean_1min,power_min_1min,power_max_1min,power_range_1min,core_temp_mean_1min,core_temp_min_1min,core_temp_max_1min,core_temp_range_1min,mem_temp_mean_1min,mem_temp_min_1min,mem_temp_max_1min,mem_temp_range_1min
0,2020-10-05 15:22:36+00:00,a06n13,3,3.238170e+11,0,NaN,NaN,0.990875,1431524.0,3964fb85353cda6319ee40094e447c3cc842b57d9e1725...,96520f52939a9005a2f723752d246aa8c355f22c849542...,4200815.0,0.000000e+00,True,False,False,False,False,False,False,False,False,False,2,0.995367,5,5.973337,7,23.955706,56,119.066958,35.0,2.0,33.0,2023,340,88,28,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.000512,0.058937,0.125505,0.239123,0.006509,0.011990,0.226833,0.000000,0.000000,0.185185,0.000000,0.000000,0.117647,0.000000,0.000000,17.620121,0.180033,0.319890,77.229276,32.989384,30.052375,10.0,419.0,409.0,24.0,53.0,29.0,21.0,56.0,35.0,0,NaN,0,NaN,0,NaN,47.320172,33.0,367.0,334.0,29.894468,28.0,37.0,9.0,26.618277,25.0,36.0,11.0,33.200920,33.0,48.0,15.0,29.955521,29.0,31.0,2.0,26.936810,25.0,28.0,3.0,33.181034,33.0,35.0,2.0,30.000000,30.0,30.0,0.0,27.000000,27.0,27.0,0.0,33.147059,33.0,34.0,1.0,30.000000,30.0,30.0,0.0,27.000000,27.0,27.0,0.0,33.071429,33.0,34.0,1.0,30.000000,30.0,30.0,0.0,27.000000,27.0,27.0,0.0
1,2020-10-05 06:41:43+00:00,a07n13,3,3.239170e+11,0,NaN,NaN,3.322851,1430887.0,fb80684aa48268896c159222c220515678b54ea6732db4...,29748b20d0924aba9d64fcfb7c9e50afd2e5ec0ebbb830...,8288790.0,0.000000e+00,False,False,False,False,False,False,False,False,False,False,1,1.000000,2,5.991718,5,23.964134,44,119.207576,79.0,45.0,34.0,1929,314,76,30,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.371020,0.008454,0.035901,0.382442,0.000000,0.013118,0.378849,0.000000,0.016064,0.334728,0.000000,0.025105,0.448980,0.000000,0.

In [4]:
print(df["is_failure"].value_counts())
print(df["is_failure"].value_counts(normalize=True))

is_failure
0    49964
1      295
Name: count, dtype: int64
is_failure
0    0.99413
1    0.00587
Name: proportion, dtype: float64


In [5]:
print(df.dtypes.head(20))

timestamp           object
hostname            object
GPU                  int64
serial             float64
is_failure           int64
prev_xid48         float64
next_xid48         float64
hours_in_job       float64
allocation_id      float64
user_name           object
account             object
gpu_energy         float64
gpu_usage          float64
flag_gpumps         object
flag_gpudefault     object
flag_smt1           object
flag_smt2           object
flag_smt4           object
flag_nvme           object
flag_cpublink       object
dtype: object


In [6]:
#checking missing values
missing = df.isnull().sum()
print("Columns with missing values:")
print(missing[missing > 0].sort_values(ascending=False).head(30))

Columns with missing values:
last_zero_power         50240
prev_xid48              49982
next_xid48              49977
last_zero_core_temp     49848
last_zero_mem_temp      49833
flag_gpumps             20913
flag_nvme               20913
flag_gpudefault         20913
flag_disableautonuma    20913
flag_cpublink           20913
flag_isolategpfs        20913
flag_maximizegpfs       20913
flag_smt2               20913
flag_smt1               20913
flag_smt4               20913
mem_temp_fluct_1min     17450
mem_temp_fluct_5min     17353
mem_temp_range_1min     17344
mem_temp_min_1min       17344
mem_temp_max_1min       17344
mem_temp_mean_1min      17344
mem_temp_range_5min     17334
mem_temp_min_5min       17334
mem_temp_max_5min       17334
mem_temp_mean_5min      17334
mem_temp_fluct_15min    17322
mem_temp_range_15min    17316
mem_temp_max_15min      17316
mem_temp_min_15min      17316
mem_temp_mean_15min     17316
dtype: int64


In [7]:
#listing object-type columns
obj_cols = df.select_dtypes(include=["object"]).columns.tolist()
print("Object columns:", obj_cols)

Object columns: ['timestamp', 'hostname', 'user_name', 'account', 'flag_gpumps', 'flag_gpudefault', 'flag_smt1', 'flag_smt2', 'flag_smt4', 'flag_nvme', 'flag_cpublink', 'flag_disableautonuma', 'flag_isolategpfs', 'flag_maximizegpfs', 'last_zero_power', 'last_zero_core_temp', 'last_zero_mem_temp']


In [8]:
# =========================
# DROP — leakage / near-empty features
# These leak failure timing or contain mostly nulls
# =========================
leakage_or_empty = [
    "prev_xid48",
    "next_xid48"
]
df = df.drop(columns=[c for c in leakage_or_empty if c in df.columns], errors="ignore")


# =========================
# DROP — identifiers with no physics meaning
# These force the model to memorize hosts or users
# =========================
id_columns = [
    "timestamp",
    "hostname",
    "serial",
    "allocation_id",
    "user_name",
    "account"
]
df = df.drop(columns=[c for c in id_columns if c in df.columns], errors="ignore")


# =========================
# DROP — categorical GPU label
# This is a device index, not behavior
# =========================
gpu_label = ["GPU"]
df = df.drop(columns=[c for c in gpu_label if c in df.columns], errors="ignore")


# =========================
# DROP — time-in-job as raw scalar
# Acts as an age proxy, not physical health
# =========================
time_proxy = ["hours_in_job"]
df = df.drop(columns=[c for c in time_proxy if c in df.columns], errors="ignore")


# =========================
# OPTIONAL DROP — scheduler and placement flags
# Software placement, not hardware condition
# Keep for system-level studies, drop for hardware-focused paper
# =========================
scheduler_flags = [
    "flag_gpumps",
    "flag_gpudefault",
    "flag_smt1",
    "flag_smt2",
    "flag_smt4",
    "flag_nvme",
    "flag_cpublink",
    "flag_disableautonuma",
    "flag_isolategpfs",
    "flag_maximizegpfs"
]
df = df.drop(columns=[c for c in scheduler_flags if c in df.columns], errors="ignore")


# =========================
# OPTIONAL DROP — workload context features
# Load context, not health signals
# Keep if modeling workload effects
# =========================
workload_context = [
    "n_jobs_1h", "jobhours_1h",
    "n_jobs_6h", "jobhours_6h",
    "n_jobs_24h", "jobhours_24h",
    "n_jobs_120h", "jobhours_120h"
]
df = df.drop(columns=[c for c in workload_context if c in df.columns], errors="ignore")


# =========================
# DROP — near-empty timestamp proxies
# Almost fully missing and time-based
# =========================
near_empty_time = [
    "last_zero_power",
    "last_zero_core_temp",
    "last_zero_mem_temp"
]
df = df.drop(columns=[c for c in near_empty_time if c in df.columns], errors="ignore")


# Quick check
print("Remaining columns:", len(df.columns))
print("Dropped leakage or empty:", [c for c in leakage_or_empty if c not in df.columns])
print("Dropped IDs:", [c for c in id_columns if c not in df.columns])
print("Dropped GPU label:", [c for c in gpu_label if c not in df.columns])
print("Dropped time proxy:", [c for c in time_proxy if c not in df.columns])
print("Dropped scheduler flags:", [c for c in scheduler_flags if c not in df.columns])
print("Dropped workload context:", [c for c in workload_context if c not in df.columns])
print("Dropped near-empty time:", [c for c in near_empty_time if c not in df.columns])


Remaining columns: 119
Dropped leakage or empty: ['prev_xid48', 'next_xid48']
Dropped IDs: ['timestamp', 'hostname', 'serial', 'allocation_id', 'user_name', 'account']
Dropped GPU label: ['GPU']
Dropped time proxy: ['hours_in_job']
Dropped scheduler flags: ['flag_gpumps', 'flag_gpudefault', 'flag_smt1', 'flag_smt2', 'flag_smt4', 'flag_nvme', 'flag_cpublink', 'flag_disableautonuma', 'flag_isolategpfs', 'flag_maximizegpfs']
Dropped workload context: ['n_jobs_1h', 'jobhours_1h', 'n_jobs_6h', 'jobhours_6h', 'n_jobs_24h', 'jobhours_24h', 'n_jobs_120h', 'jobhours_120h']
Dropped near-empty time: ['last_zero_power', 'last_zero_core_temp', 'last_zero_mem_temp']


In [9]:
# Variance per feature
variance = df.var(numeric_only=True)

# Exact constant features (zero variance)
constant_features = variance[variance == 0].index.tolist()

print("Constant features (drop these):")
print(constant_features)


# =========================
# NEAR-ZERO VARIANCE CHECK
# Drops features with almost no variation
# =========================
threshold = 1e-6
near_zero = variance[variance < threshold].index.tolist()

print("\nNear-zero variance features (review or drop):")
print(near_zero)


# =========================
# DROP THEM
# =========================
to_drop = list(set(constant_features + near_zero))
df = df.drop(columns=to_drop, errors="ignore")

print("\nDropped", len(to_drop), "low-variance features")
print("New shape:", df.shape)


# =========================
# VERIFY SCALING
# Mean should be near 0 and std near 1 for scaled data
# =========================
means = df.mean(numeric_only=True)
stds = df.std(numeric_only=True)

print("\nScaling check (first 10 features):")
print(pd.DataFrame({
    "mean": means.head(10),
    "std": stds.head(10)
}))

print("\nAverage std across features:", stds.mean())
print("Min std across features:", stds.min())


Constant features (drop these):
[]

Near-zero variance features (review or drop):
[]

Dropped 0 low-variance features
New shape: (50259, 119)

Scaling check (first 10 features):
                         mean           std
is_failure       5.869595e-03  7.638887e-02
gpu_energy       1.536652e+07  1.776609e+07
gpu_usage        4.893239e+10  8.281762e+10
job_max_power    1.542611e+02  1.333760e+02
job_range_power  1.262064e+02  1.271323e+02
job_min_power    2.805472e+01  1.085145e+01
n_no_tele_6h     3.449953e+03  5.385213e+03
n_no_tele_1h     5.718142e+02  9.228807e+02
n_no_tele_15min  1.427299e+02  2.320695e+02
n_no_tele_5min   4.756935e+01  7.756307e+01

Average std across features: 696096026.0003288
Min std across features: 0.019273957893198402


In [8]:
# STAGE 1 — SAVE CLEANED DATASET
clean_path = "datasets/new_summit_gpu_clean.parquet"
df.to_parquet(clean_path, index=False)
print("Saved cleaned dataset:", clean_path)

Saved cleaned dataset: datasets/new_summit_gpu_clean.parquet


In [11]:
# take one real row from your dataset
row = df.drop(columns=["is_failure"]).iloc[[0]]

# save it ready for Gradio
row.to_csv("demo_row_real.csv", index=False)

print("Saved: demo_row_real.csv")
print(row.head())

Saved: demo_row_real.csv
   gpu_energy  gpu_usage  job_max_power  job_range_power  job_min_power  n_no_tele_6h  n_no_tele_1h  n_no_tele_15min  n_no_tele_5min  n_no_tele_1min  n_NA_power_6h  n_NA_core_temp_6h  \
0   4200815.0        0.0           35.0              2.0           33.0          2023           340               88              28               4              0                  0   

   n_NA_mem_temp_6h  n_NA_power_1h  n_NA_core_temp_1h  n_NA_mem_temp_1h  n_NA_power_15min  n_NA_core_temp_15min  n_NA_mem_temp_15min  n_NA_power_5min  n_NA_core_temp_5min  n_NA_mem_temp_5min  \
0                 0              0                  0                 0                 0                     0                    0                0                    0                   0   

   n_NA_power_1min  n_NA_core_temp_1min  n_NA_mem_temp_1min  power_fluct_6h  core_temp_fluct_6h  mem_temp_fluct_6h  power_fluct_1h  core_temp_fluct_1h  mem_temp_fluct_1h  power_fluct_15min  \
0                0   